# 东方财富网国内新闻数据抓取器
## 拂晓工作室
### 此程序将爬取某个板块所有最近的新闻并存进CSV！

# 参考资料
* [1] [BeautifulSoup的例子](https://qiita.com/rusarusa/items/d7f014ba80d6fe7a3e07)
* [2] [BeautifulSoup的文档](https://www.cnblogs.com/zhaof/p/6930955.html)
* [3] [暴力安装BS4](https://github.com/ipython/ipython/issues/10684)
* [4] [CSV文档](https://docs.python.org/2/library/csv.html)
* [5] [CSV使用实例](https://blog.csdn.net/waple_0820/article/details/70049953)
* [6] [如何解决list重复元素的问题](https://stackoverflow.com/questions/7961363/removing-duplicates-in-lists)
* [7] [解决CSV文件中文乱码的问题](https://blog.csdn.net/fwj_ntu/article/details/78563962)
* [8] [在macOS下解决中文乱码问题](http://blog.sina.com.cn/s/blog_69e225900102w2mc.html)

In [1]:
# 导入库
import urllib
import re
import pandas as pd
import pymysql
import os
import turicreate as tc
import csv
from bs4 import BeautifulSoup

/Users/hongjunwu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 新闻板块(同时也会加入热门新闻的数据)
china = 'cgnjj'  # 国内
international = 'cgjjj'  # 国际

# 定义基本常量
url = 'http://finance.eastmoney.com/news/' + china + '.html' # 主目录地址 更换不同主目录以爬取不同板块内容 (BAD)
csv_filepath = '../Datasets/Eastmoney/news/'  # 存储数据文件地址

In [3]:
# 创建一个二十五页的list
page_list = []
counter = 1

while counter <= 25:
    pageurl = 'http://finance.eastmoney.com/news/' + china
    if counter != 1:
        pageurl = pageurl + '_' + str(counter) + '.html'
        page_list.append(pageurl)
    else:
        pageurl = pageurl + '.html'
        page_list.append(pageurl)
    counter += 1
    
print('成功创建包含 ' + str(len(page_list)) + ' 个页面链接的目录！')

成功创建包含 25 个页面链接的目录！


In [4]:
# 创建一个包含20个class的list
counter = 0
class_list = []

while counter < 20:
    class_list.append('newsTr' + str(counter))
    counter += 1
    
print('成功创建包含 ' + str(len(class_list)) + ' 个class的目录！')
# print(class_list)  # Debug

成功创建包含 20 个class的目录！


In [5]:
# 初始化urllist
urllist = []
year = '2018'  # Separate parameter
counter = 1

# 创造urllist
print('-----------------获取所有文章超链接程序-----------------')
print('分析任务开始！')
for url in page_list:
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    for each_url in soup.find_all('a', href=True):
        if 'http://finance.eastmoney.com/news/' in each_url['href']:
            if year in each_url['href']:
                urllist.append(each_url['href'])   
    print('第' + str(counter) + '页分析完成！现在一共有' + str(len(urllist)) + '篇文章的数据！')
    counter += 1

urllist = list(set(urllist))
print('全部分析完成！正在查重...')
print('任务完成！共获得了' + str(len(urllist)) + '篇文章的数据！')
print('---------------------------------------------')

-----------------获取所有文章超链接程序-----------------
分析任务开始！
第1页分析完成！现在一共有37篇文章的数据！
第2页分析完成！现在一共有74篇文章的数据！
第3页分析完成！现在一共有115篇文章的数据！
第4页分析完成！现在一共有151篇文章的数据！
第5页分析完成！现在一共有187篇文章的数据！
第6页分析完成！现在一共有230篇文章的数据！
第7页分析完成！现在一共有269篇文章的数据！
第8页分析完成！现在一共有307篇文章的数据！
第9页分析完成！现在一共有343篇文章的数据！
第10页分析完成！现在一共有380篇文章的数据！
第11页分析完成！现在一共有416篇文章的数据！
第12页分析完成！现在一共有453篇文章的数据！
第13页分析完成！现在一共有491篇文章的数据！
第14页分析完成！现在一共有532篇文章的数据！
第15页分析完成！现在一共有570篇文章的数据！
第16页分析完成！现在一共有607篇文章的数据！
第17页分析完成！现在一共有644篇文章的数据！
第18页分析完成！现在一共有681篇文章的数据！
第19页分析完成！现在一共有717篇文章的数据！
第20页分析完成！现在一共有753篇文章的数据！
第21页分析完成！现在一共有795篇文章的数据！
第22页分析完成！现在一共有838篇文章的数据！
第23页分析完成！现在一共有874篇文章的数据！
第24页分析完成！现在一共有910篇文章的数据！
第25页分析完成！现在一共有946篇文章的数据！
全部分析完成！正在查重...
任务完成！共获得了516篇文章的数据！
---------------------------------------------


In [11]:
# 抓取新闻URL函数
def collectNews(url, counter):
    # 准备工作
    req = urllib.request.Request(url)
    print('请求链接连接成功!')
    response = urllib.request.urlopen(req)
    print('收到反馈信号！')
    html = response.read()
    print('HTML生成完毕！')
    soup = BeautifulSoup(html, "lxml")
    print('SOUP创建完成！')
    
    # 获得文章的发表时间 
    time = soup.find(class_="time").get_text()
    print('时间获取完毕！')
    
    # 获得文章的内容
    content = soup.find(id="ContentBody").get_text() 
    content = content[0:re.search('责任编辑', content).span()[0]-1].strip()
    print('文章内容获取完成！')
    
    # 获得相关的股票单
    related_stocks = []
    for each in soup.find_all(class_='keytip'): 
        related_stocks.append(each.get_text())
    print('相关内容单获取完成！')   
        
    # 写入csv
    with open("../DataSets/Eastmoney/news/RecentStockNews.csv", "a") as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow([counter, 
                         time[0:4], time[5:8], time[8:10], 
                         soup.find('h1').get_text(), 
                         content, 
                         related_stocks])
    print('csv写入完成！')
    
    # 释放内存
    del(req, response, html, soup, time, content, related_stocks)
    
    # 刷新计数 
    counter += 1
    print('计数刷新完成！')
    print('页面数据获取完毕！')
    return counter

In [12]:
# 创建csv文件
with open("../DataSets/Eastmoney/news/RecentStockNews.csv", "w") as csvfile: 
    writer = csv.writer(csvfile)

    # 先写入columns_name
    writer.writerow(["index", "year", "month", "day", "title", "contents", "related"])

In [13]:
# 初始化计数器
counter = 1

# 获取总任务数
total = len(urllist)

# 下载数据
for each_article in urllist:
    print('=============================================')
    print('正在获取第' + str(counter) + '篇文章.')
    print('--------------------------------------------------------')
    counter = collectNews(each_article, counter) 
    
print('获取完毕！共获取了' + str(total) + '篇文章!')

正在获取第1篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第2篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第3篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第4篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第5篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第6篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第7篇文章.
-----------------

收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第41篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第42篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第43篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第44篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第45篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第46篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP

收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第81篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第82篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第83篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第84篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第85篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第86篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP

收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第121篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第122篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第123篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第124篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第125篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第126篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕

HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第160篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第161篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第162篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第163篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第164篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第165篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建

...

收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第358篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第359篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第360篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第361篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第362篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第363篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕

HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第397篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第398篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第399篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第400篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第401篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第402篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建

收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第437篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第438篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第439篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第440篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第441篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第442篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕

HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第476篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第477篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第478篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第479篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第480篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第481篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建

收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
正在获取第516篇文章.
--------------------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
csv写入完成！
计数刷新完成！
页面数据获取完毕！
获取完毕！共获取了516篇文章!
